# CNNs desde Cero - Operaciones y Entrenamiento

**Objetivo:** En este notebook especial, vamos a desmitificar por completo las Redes Neuronales Convolucionales. Dejaremos de lado TensorFlow y construiremos las operaciones fundamentales desde cero usando únicamente NumPy. 

1.  **Parte 1:** Ilustraremos el `forward pass` de una única capa convolucional para entender mecánicamente cómo funcionan el padding, la convolución, la activación y el pooling.
2.  **Parte 2:** Simularemos el entrenamiento de una mini-CNN completa para ver cómo la red puede *aprender* los filtros correctos a través de la retropropagación y el descenso de gradiente.

## Parte 1: Anatomía de una Capa Convolucional (Forward Pass)

En esta primera parte, nos enfocamos en visualizar qué ocurre dentro de una capa `Conv2D` cuando procesa una imagen. La siguiente clase, `CapaConvolucionalManual`, implementa cada paso de forma explícita.

In [1]:
import numpy as np

In [2]:
class CapaConvolucionalManual:
    """
    Una clase para demostrar el funcionamiento de una capa convolucional
    y de pooling desde cero, utilizando únicamente NumPy.
    """
    def __init__(self, kernel, stride=1, padding=0, pool_size=2, pool_stride=2):
        """
        Inicializa la capa con sus hiperparámetros.

        :param kernel: El filtro (o kernel) a aplicar. Debe ser un array 2D de NumPy.
        :type kernel: np.ndarray
        :param stride: El paso (stride) del filtro al convolucionar.
        :type stride: int
        :param padding: El ancho del relleno (padding) a añadir a la imagen.
        :type padding: int
        :param pool_size: El tamaño de la ventana de pooling (ej. 2 para una ventana de 2x2).
        :type pool_size: int
        :param pool_stride: El paso (stride) de la ventana de pooling.
        :type pool_stride: int
        """
        self.kernel = kernel
        self.stride = stride
        self.padding = padding
        self.pool_size = pool_size
        self.pool_stride = pool_stride
        print("Capa inicializada con éxito.")
        print(f"Kernel (Filtro):\n{self.kernel}")
        print(f"Stride: {self.stride}, Padding: {self.padding}")
        print(f"Tamaño de Pooling: {self.pool_size}, Stride de Pooling: {self.pool_stride}\n")

    def _aplicar_padding(self, imagen):
        if self.padding == 0:
            return imagen
        return np.pad(imagen, pad_width=self.padding, mode='constant', constant_values=0)

    def _operacion_convolucion(self, region_imagen):
        producto = self.kernel * region_imagen
        return np.sum(producto)

    def convolucionar(self, imagen_padded):
        alto_kernel, ancho_kernel = self.kernel.shape
        alto_imagen, ancho_imagen = imagen_padded.shape
        alto_salida = (alto_imagen - alto_kernel) // self.stride + 1
        ancho_salida = (ancho_imagen - ancho_kernel) // self.stride + 1
        feature_map = np.zeros((alto_salida, ancho_salida))
        for y in range(0, alto_salida):
            for x in range(0, ancho_salida):
                y_inicio = y * self.stride
                x_inicio = x * self.stride
                region = imagen_padded[y_inicio : y_inicio + alto_kernel, x_inicio : x_inicio + ancho_kernel]
                feature_map[y, x] = self._operacion_convolucion(region)
        return feature_map

    def _relu(self, feature_map):
        return np.maximum(0, feature_map)

    def _max_pooling(self, feature_map_activado):
        alto_mapa, ancho_mapa = feature_map_activado.shape
        alto_salida = (alto_mapa - self.pool_size) // self.pool_stride + 1
        ancho_salida = (ancho_mapa - self.pool_size) // self.pool_stride + 1
        pooled_map = np.zeros((alto_salida, ancho_salida))
        for y in range(0, alto_salida):
            for x in range(0, ancho_salida):
                y_inicio = y * self.pool_stride
                x_inicio = x * self.pool_stride
                region = feature_map_activado[y_inicio : y_inicio + self.pool_size, x_inicio : x_inicio + self.pool_size]
                pooled_map[y, x] = np.max(region)
        return pooled_map

    def forward_pass(self, imagen):
        print("--- INICIANDO FORWARD PASS ---")
        print("1. Aplicando Padding...")
        imagen_padded = self._aplicar_padding(imagen)
        print(f"Imagen Original:\n{imagen}")
        print(f"Imagen con Padding (tamaño {self.padding}):\n{imagen_padded}\n")
        print("2. Aplicando Convolución...")
        feature_map = self.convolucionar(imagen_padded)
        print(f"Mapa de Características (Feature Map) resultante:\n{feature_map}\n")
        print("3. Aplicando Activación ReLU...")
        feature_map_activado = self._relu(feature_map)
        print(f"Mapa de Características Activado (valores negativos a 0):\n{feature_map_activado}\n")
        print("4. Aplicando Max Pooling...")
        pooled_map = self._max_pooling(feature_map_activado)
        print(f"Mapa de Características Final (después de Pooling):\n{pooled_map}\n")
        print("--- FORWARD PASS COMPLETADO ---")
        return pooled_map

In [3]:
# --- Ejecución de la Parte 1 ---
print("### DEMOSTRACIÓN DE LA CAPA CONVOLUCIONAL MANUAL ###\n")
imagen_ejemplo = np.array([
    [10, 10, 10,  0,  0],
    [10, 10, 10,  0,  0],
    [10, 10, 10,  0,  0],
    [ 0,  0,  0, 10, 10],
    [ 0,  0,  0, 10, 10]
])

filtro_ejemplo = np.array([
    [ 1,  0, -1],
    [ 1,  0, -1],
    [ 1,  0, -1]
])

capa_cnn = CapaConvolucionalManual(kernel=filtro_ejemplo, stride=1, padding=1, pool_size=2, pool_stride=2)
resultado_final = capa_cnn.forward_pass(imagen_ejemplo)

### DEMOSTRACIÓN DE LA CAPA CONVOLUCIONAL MANUAL ###

Capa inicializada con éxito.
Kernel (Filtro):
[[ 1  0 -1]
 [ 1  0 -1]
 [ 1  0 -1]]
Stride: 1, Padding: 1
Tamaño de Pooling: 2, Stride de Pooling: 2

--- INICIANDO FORWARD PASS ---
1. Aplicando Padding...
Imagen Original:
[[10 10 10  0  0]
 [10 10 10  0  0]
 [10 10 10  0  0]
 [ 0  0  0 10 10]
 [ 0  0  0 10 10]]
Imagen con Padding (tamaño 1):
[[ 0  0  0  0  0  0  0]
 [ 0 10 10 10  0  0  0]
 [ 0 10 10 10  0  0  0]
 [ 0 10 10 10  0  0  0]
 [ 0  0  0  0 10 10  0]
 [ 0  0  0  0 10 10  0]
 [ 0  0  0  0  0  0  0]]

2. Aplicando Convolución...
Mapa de Características (Feature Map) resultante:
[[-20.   0.  20.  20.   0.]
 [-30.   0.  30.  30.   0.]
 [-20.   0.  10.  10.  10.]
 [-10.   0. -10. -10.  20.]
 [  0.   0. -20. -20.  20.]]

3. Aplicando Activación ReLU...
Mapa de Características Activado (valores negativos a 0):
[[ 0.  0. 20. 20.  0.]
 [ 0.  0. 30. 30.  0.]
 [ 0.  0. 10. 10. 10.]
 [ 0.  0.  0.  0. 20.]
 [ 0.  0.  0.  0. 20.]]

4. Apl

## Parte 2: Entrenamiento de una Mini-CNN Completa

Ahora que entendemos las operaciones, vamos a unirlas en una red completa y a simular cómo "aprende". Crearemos un dataset sintético donde el objetivo es clasificar si una imagen contiene una línea vertical u horizontal. Veremos cómo la red, partiendo de un filtro aleatorio, ajusta sus pesos para resolver la tarea.

In [4]:
def generar_dataset(n_muestras):
    """
    Genera un dataset sintético de imágenes 5x5.
    """
    X = []
    y = []
    for i in range(n_muestras):
        imagen = np.zeros((5, 5))
        if i % 2 == 0:
            # Línea vertical (target = 1)
            imagen[:, 2] = 1
            y.append(1)
        else:
            # Línea horizontal (target = 0)
            imagen[2, :] = 1
            y.append(0)
        imagen += np.random.rand(5, 5) * 0.2
        X.append(imagen)
    return np.array(X), np.array(y).reshape(-1, 1)

In [5]:
class MiniCNNManual:
    """
    Una clase que simula una CNN completa con una capa convolucional,
    pooling, y una capa densa para clasificación binaria.
    Implementa el entrenamiento desde cero.
    """
    def __init__(self, learning_rate=0.01):
        self.kernel = np.random.randn(3, 3) * 0.1
        self.dense_weights = np.random.randn(4, 1) * 0.1
        self.dense_bias = np.random.randn(1, 1) * 0.1
        self.lr = learning_rate
        print("Mini-CNN inicializada.")
        print(f"Kernel inicial (aleatorio):\n{self.kernel}")

    def _sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def _relu(self, x):
        return np.maximum(0, x)

    def _binary_cross_entropy_loss(self, y_true, y_pred):
        return - (y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred))

    def forward(self, imagen):
        conv_output = np.zeros((3, 3))
        for y in range(3):
            for x in range(3):
                region = imagen[y:y+3, x:x+3]
                conv_output[y, x] = np.sum(region * self.kernel)
        relu_output = self._relu(conv_output)
        pool_output = np.zeros((2, 2))
        for y in range(2):
            for x in range(2):
                region = relu_output[y:y+2, x:x+2]
                pool_output[y, x] = np.max(region)
        flattened_output = pool_output.flatten().reshape(1, -1)
        dense_input = flattened_output
        dense_output = dense_input @ self.dense_weights + self.dense_bias
        prediction = self._sigmoid(dense_output)
        cache = {
            "imagen": imagen, "conv_output": conv_output, "relu_output": relu_output,
            "pool_output": pool_output, "dense_input": dense_input, "prediction": prediction
        }
        return prediction, cache

    def backward(self, y_true, cache):
        d_loss_d_pred = - (y_true / cache["prediction"] - (1 - y_true) / (1 - cache["prediction"]))
        d_pred_d_dense = cache["prediction"] * (1 - cache["prediction"])
        d_loss_d_dense = d_loss_d_pred * d_pred_d_dense
        self.grad_dense_weights = cache["dense_input"].T @ d_loss_d_dense
        self.grad_dense_bias = np.sum(d_loss_d_dense, axis=0, keepdims=True)
        d_loss_d_flatten = d_loss_d_dense @ self.dense_weights.T
        d_loss_d_pool = d_loss_d_flatten.reshape(cache["pool_output"].shape)
        d_loss_d_relu = np.zeros(cache["relu_output"].shape)
        for y in range(2):
            for x in range(2):
                region = cache["relu_output"][y:y+2, x:x+2]
                max_val = np.max(region)
                mask = (region == max_val)
                d_loss_d_relu[y:y+2, x:x+2] += d_loss_d_pool[y,x] * mask
        d_relu_d_conv = cache["relu_output"] > 0
        d_loss_d_conv = d_loss_d_relu * d_relu_d_conv
        self.grad_kernel = np.zeros(self.kernel.shape)
        for y in range(3):
            for x in range(3):
                region = cache["imagen"][y:y+3, x:x+3]
                self.grad_kernel += region * d_loss_d_conv[y, x]

    def train(self, X, y, epochs):
        print("\n--- INICIANDO ENTRENAMIENTO ---")
        for epoch in range(epochs):
            total_loss = 0
            for imagen, etiqueta in zip(X, y):
                prediccion, cache = self.forward(imagen)
                total_loss += self._binary_cross_entropy_loss(etiqueta, prediccion)
                self.backward(etiqueta, cache)
                self.kernel -= self.lr * self.grad_kernel
                self.dense_weights -= self.lr * self.grad_dense_weights
                self.dense_bias -= self.lr * self.grad_dense_bias
            if (epoch + 1) % 10 == 0:
                avg_loss = total_loss / len(X)
                print(f"Época {epoch+1}/{epochs}, Pérdida Promedio: {avg_loss[0][0]:.4f}")
        print("--- ENTRENAMIENTO COMPLETADO ---")
        print(f"Kernel final (aprendido):\n{self.kernel}")

In [6]:
# --- Ejecución de la Parte 2 ---
print("\n### SIMULACIÓN DE ENTRENAMIENTO DE MINI-CNN ###\n")
X_train, y_train = generar_dataset(100)
cnn = MiniCNNManual(learning_rate=0.01)
cnn.train(X_train, y_train, epochs=50)

print("\n--- PROBANDO MODELO ENTRENADO ---")
test_vertical = np.zeros((5,5))
test_vertical[:, 2] = 1

test_horizontal = np.zeros((5,5))
test_horizontal[2, :] = 1

pred_v, _ = cnn.forward(test_vertical)
pred_h, _ = cnn.forward(test_horizontal)

print(f"\nImagen de prueba (Línea Vertical):\n{test_vertical}")
print(f"Predicción (debería ser cercano a 1): {pred_v[0][0]:.4f}")

print(f"\nImagen de prueba (Línea Horizontal):\n{test_horizontal}")
print(f"Predicción (debería ser cercano a 0): {pred_h[0][0]:.4f}")


### SIMULACIÓN DE ENTRENAMIENTO DE MINI-CNN ###

Mini-CNN inicializada.
Kernel inicial (aleatorio):
[[-0.02355612  0.00804416  0.03578015]
 [ 0.05786005 -0.01891272 -0.03601907]
 [-0.05485257  0.01644713  0.09190044]]

--- INICIANDO ENTRENAMIENTO ---


Época 10/50, Pérdida Promedio: 0.2841


Época 20/50, Pérdida Promedio: 0.0348


Época 30/50, Pérdida Promedio: 0.0111


Época 40/50, Pérdida Promedio: 0.0060


Época 50/50, Pérdida Promedio: 0.0039
--- ENTRENAMIENTO COMPLETADO ---
Kernel final (aprendido):
[[-0.60901178  0.76119948 -0.55159816]
 [-0.50227857  0.73741483 -0.6180919 ]
 [ 0.41475851  1.85295713  0.57305541]]

--- PROBANDO MODELO ENTRENADO ---

Imagen de prueba (Línea Vertical):
[[0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0.]]
Predicción (debería ser cercano a 1): 0.9955

Imagen de prueba (Línea Horizontal):
[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [1. 1. 1. 1. 1.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]
Predicción (debería ser cercano a 0): 0.0066


### Conclusión del Notebook

En este notebook hemos construido una CNN desde sus cimientos. Vimos cómo las operaciones matemáticas de convolución y pooling transforman una imagen, y lo más importante, cómo el proceso de entrenamiento (forward y backward pass) permite a la red **aprender** los valores correctos para sus filtros, especializándose en la tarea que le hemos asignado. Este entendimiento fundamental es clave para usar eficazmente las librerías de alto nivel como TensorFlow y Keras.